In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
os.chdir('data')

In [5]:
data_path = os.getcwd()

In [6]:
train_cifar = datasets.CIFAR10(data_path, train=True, download=True, 
                             transform = transforms.Compose([
                                 transforms.ToTensor()
                                                            ]))

Files already downloaded and verified


In [7]:
val_cifar = datasets.CIFAR10(data_path, train=False, download=True,
                             transform = transforms.Compose([
                                 transforms.ToTensor()
                                                            ]))

Files already downloaded and verified


In [8]:
train_loader = torch.utils.data.DataLoader(train_cifar, batch_size=64, shuffle=True)

In [9]:
val_loader = torch.utils.data.DataLoader(val_cifar, batch_size=64, shuffle=True)

In [10]:
def accuracy(loader, model):
    if loader.dataset.train:
        print("checking accuracy on training set:")
    else:
        print("checking accuracy on validation set:")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to('cuda')
            labels = labels.to('cuda')
            
            imgs = imgs.squeeze(1)
            scores = model(imgs)
            
            _, predictions = torch.max(scores, dim=1)
            num_correct += (predictions==labels).sum()
            num_samples += predictions.size(0)
        print(f'got {num_correct} / {num_samples} with accuracy \
               {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

In [51]:
imgs, labels = next(iter(train_loader))

In [56]:
imgs.squeeze(1).shape

torch.Size([64, 28, 28])

In [61]:
imgs.squeeze(1).size(0)

64

In [70]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.RNN(input_size = 28, hidden_size = 256, num_layers=2, batch_first=True)
        
        self.fc = nn.Linear(256*28, 10)
        
    def forward(self, t):
        h0 = torch.zeros(2, t.size(0), 256).to('cuda')
        
        #forward pass:
        out, _ = self.rnn(t, h0)
        out = out.reshape(out.shape[0], -1)
        return self.fc(out)

In [71]:
model = RNN().to('cuda')

optimizer = optim.Adam(model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [11]:
def training_loop(n_epochs, model, optimizer, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        train_loss = 0.0
        num_samples = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to('cuda')
            labels = labels.to('cuda')
            
            imgs = imgs.squeeze(1)
            
            scores = model(imgs)
            
            loss = loss_fn(scores, labels)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            train_loss += loss
            num_samples += scores.size(0)
        
        print(f'{epoch} \t loss: {loss / num_samples}')

In [73]:
training_loop(n_epochs = 2,
              model = model,
              optimizer = optimizer,
              loss_fn = loss_fn,
              train_loader = train_loader)

1 	 loss: 2.434250518490444e-06
2 	 loss: 1.3552023574447958e-06


In [76]:
accuracy(loader=train_loader, model=model)
accuracy(loader=val_loader, model=model)

checking accuracy on training set:
got 57909 / 60000 with accuracy                96.52
checking accuracy on validation set:
got 9617 / 10000 with accuracy                96.17


In [79]:
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        
        self.gru = nn.GRU(input_size = 28, hidden_size = 256, num_layers=2, batch_first=True)
        
        self.fc = nn.Linear(256*28, 10)
        
    def forward(self, t):
        h0 = torch.zeros(2, t.size(0), 256).to('cuda')
        
        #forward pass:
        out, _ = self.gru(t, h0)
        out = out.reshape(out.shape[0], -1)
        return self.fc(out)

In [80]:
gru_model = GRU().to('cuda')

gru_optimizer = optim.Adam(gru_model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [83]:
training_loop(n_epochs = 2,
              model = gru_model,
              optimizer = gru_optimizer,
              loss_fn = loss_fn,
              train_loader = train_loader)

1 	 loss: 2.595330443000421e-06
2 	 loss: 2.255755333635534e-07


In [84]:
accuracy(loader=train_loader, model=gru_model)
accuracy(loader=val_loader, model=gru_model)

checking accuracy on training set:
got 59093 / 60000 with accuracy                98.49
checking accuracy on validation set:
got 9854 / 10000 with accuracy                98.54


In [85]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size = 28, hidden_size = 256, num_layers=2, batch_first=True)
        
        self.fc = nn.Linear(256*28, 10)
        
    def forward(self, t):
        h0 = torch.zeros(2, t.size(0), 256).to('cuda')
        c0 = torch.zeros(2, t.size(0), 256).to('cuda')
        
        #forward pass:
        out, _ = self.lstm(t, (h0, c0))
        out = out.reshape(out.shape[0], -1)
        return self.fc(out)

In [89]:
lstm_model = LSTM().to('cuda')

lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [90]:
training_loop(n_epochs = 2,
              model = lstm_model,
              optimizer = lstm_optimizer,
              loss_fn = loss_fn,
              train_loader = train_loader)

1 	 loss: 5.173962449589453e-07
2 	 loss: 3.742273690932052e-08


In [91]:
accuracy(loader=train_loader, model=lstm_model)
accuracy(loader=val_loader, model=lstm_model)

checking accuracy on training set:
got 59085 / 60000 with accuracy                98.47
checking accuracy on validation set:
got 9839 / 10000 with accuracy                98.39


In [12]:
import torchvision

In [31]:
vgg16_model = torchvision.models.vgg16(pretrained=True)
print(vgg16_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [32]:
for param in vgg16_model.parameters():
    param.requires_grad = False

In [34]:
vgg16_model.classifier[6] = nn.Linear(4096, 10)

In [43]:
vgg16_model.classifier[6]

Linear(in_features=4096, out_features=10, bias=True)

In [45]:
model = vgg16_model.to('cuda')

In [46]:
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [47]:
training_loop(n_epochs=2, model=model, optimizer=optimizer, 
             loss_fn = loss_fn, train_loader = train_loader)

1 	 loss: 2.4795570425339974e-05
2 	 loss: 2.790836697386112e-05


In [48]:
accuracy(train_loader, model)
accuracy(val_loader, model)

checking accuracy on training set:
got 28687 / 50000 with accuracy                57.37
checking accuracy on validation set:
got 5508 / 10000 with accuracy                55.08
